In [ ]:
# Try number one using onlne Dagshub Server to recieve data
# id: clearedcto
# pass: pass
# project: hello-world
#
# On your own system where you can bring up a server instructions at
#    - https://stackoverflow.com/questions/61615818/setting-up-mlflow-on-google-colab
#
# Complete DAGsHub with DVC at
#    - https://colab.research.google.com/drive/1JJIwAH0TBSY49um5s2FD0GEA6bw3SKrd?usp=sharing#scrollTo=uv3k5NWh38FZ
#

!pip install mlflow --quiet
#!databricks configure --host https://community.cloud.databricks.com/

import mlflow
import os
from getpass import getpass

os.environ['MLFLOW_TRACKING_USERNAME'] = "clearedcto" #input('Enter your DAGsHub username: ')
os.environ['MLFLOW_TRACKING_PASSWORD'] = "3d26a2125f4305b7520f19da226680944b06d65b" #getpass('Enter your DAGsHub access token: ')
os.environ['MLFLOW_TRACKING_PROJECTNAME'] = "hello-world" #input('Enter your DAGsHub project name: ')

#mlflow.set_tracking_uri("databricks")
#mlflow.set_experiment("/Users/brucegoldgmu@gmail.com/bhg_mlflow1")

mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME'] + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')

with mlflow.start_run(run_name="MLflow to Colab direct test"):
  mlflow.log_metric("m4", 4.0)
  mlflow.log_param("p4", "mlflow-colab4")

In [ ]:
# Try number two using simple KNN example from Medium Artictle
# Provides straight logging using code to local drive
#
# at: https://medium.com/p/ad1a1d8b6dcd
# code: https://github.com/bgoldfe2/starting_out_with_mlflow
# date: Jan 17, 2023
#

%cd drive/MyDrive/Github/starting_out_with_mlflow/

!python starter_code.py


In [ ]:
# Run the MLFlow version

!python mlflow_knn.py

# Refresh your folder structure to see the "mlruns" folder

In [ ]:
# Change a parameter and do a second run

!python mlflow_knn.py 3

# View the params folder to see the new value of '3'

In [3]:
# Prerequisites for article at:
# https://towardsdatascience.com/an-intuitive-guide-to-track-your-ml-experiments-with-mlflow-7ac50e63b09
#
!pip install mlflow --quiet
!pip install wget --quiet

In [ ]:
# Download and store the data set in a folder called data
%cd /content/drive/MyDrive/Github/data/

import wget
zip_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip"
filename = wget.download(zip_url)

In [ ]:
# random forest example with regular logging and MLFlow logging

import os
import warnings
import sys
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from urllib.parse import urlparse
import wget
import mlflow
import mlflow.sklearn
import logging
from zipfile import ZipFile

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

def preprocessing(df):
    X = df.drop(['instant','dteday','atemp','casual','registered','cnt'],axis=1).values
    y = df['cnt'].values
    return train_test_split(X, y, test_size=0.2)

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2    
      
def run_the_alg(n, feat, dep):
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    try:
        with ZipFile('Bike-Sharing-Dataset.zip',"r") as z:
            with z.open("hour.csv") as f:
                df = pd.read_csv(f,delimiter=',')
                display(df.head())
               
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )
    X_train, X_test, y_train, y_test = preprocessing(df)

    n_estimators = n
    max_features = feat
    max_depth = dep

    with mlflow.start_run():
        rf = RandomForestRegressor(n_estimators=n_estimators,max_features=max_features,max_depth=max_depth)
        model = rf.fit(X_train,y_train)
        y_pred = model.predict(X_test)

        (rmse, mae, r2) = eval_metrics(y_test, y_pred)
        print("Random Forest model (n_estimators=%d, max_features=%d, max_depth=%d):" % (n_estimators, max_features,max_depth))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2) 

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_features", max_features)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)  
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        if tracking_url_type_store != "file":
            mlflow.sklearn.log_model(rf, "model", registered_model_name="RandomForestModel")
        else:
            mlflow.sklearn.log_model(rf, "model") 

In [ ]:
# run_the_alg(100, 6, 5)

# also try 200, 8, 6
run_the_alg(200, 8, 6)

In [ ]:
# Databricks tutorial Getting Started Example - Wine data set

%cd /content/drive/MyDrive/Github/mlflow/examples/

!python sklearn_elasticnet_wine/train.py

/content/drive/MyDrive/Github/mlflow/examples
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


In [ ]:
!databricks configure --host https://community.cloud.databricks.com/

Databricks Host (should begin with https://): https://community.cloud.databricks.com/
Username: brucegoldgmu@gmail.com
Password: 
Repeat for confirmation: 


In [ ]:
import mlflow
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/brucegoldgmu@gmail.com/bhg_mlflow1")

In [9]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
X, y = data.data, data.target
seed = 10 #Specify a seed value.

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = seed, stratify = y)

In [15]:
from mlflow import MlflowClient

def fetch_logged_data(run_id):
    client = MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

In [19]:
from sklearn.ensemble import RandomForestClassifier

current_run = mlflow.start_run()

#Logging the seed value to passed to the train_test_split function. 
mlflow.log_param("seed", seed)

estimators = int(input("Estimator(s): "))

#Model definition.
rclf = RandomForestClassifier(n_estimators = estimators)

mlflow.sklearn.autolog()
rclf.fit(X_train, y_train)
#metrics = mlflow.sklearn.eval_and_log_metrics(rclf, X_test, y_test, prefix="val_")
params, metrics, tags, artifacts = fetch_logged_data(current_run.info.run_id) #(run.info.run_id)
mlflow.end_run()

Estimator(s): 4


In [25]:
from pprint import pprint

import pandas as pd
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

import mlflow
from mlflow.tracking import MlflowClient


def yield_artifacts(run_id, path=None):
    """Yield all artifacts in the specified run"""
    client = MlflowClient()
    for item in client.list_artifacts(run_id, path):
        if item.is_dir:
            yield from yield_artifacts(run_id, item.path)
        else:
            yield item.path


def fetch_logged_data(run_id):
    """Fetch params, metrics, tags, and artifacts in the specified run"""
    client = MlflowClient()
    data = client.get_run(run_id).data
    # Exclude system tags: https://www.mlflow.org/docs/latest/tracking.html#system-tags
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = list(yield_artifacts(run_id))
    return {
        "params": data.params,
        "metrics": data.metrics,
        "tags": tags,
        "artifacts": artifacts,
    }


def do_grid_search():
    mlflow.sklearn.autolog()

    iris = datasets.load_iris()
    parameters = {"kernel": ("linear", "rbf"), "C": [1, 10]}
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters)

    clf.fit(iris.data, iris.target)
    run_id = mlflow.last_active_run().info.run_id

    # show data logged in the parent run
    print("========== parent run ==========")
    for key, data in fetch_logged_data(run_id).items():
        print("\n---------- logged {} ----------".format(key))
        pprint(data)

    # show data logged in the child runs
    filter_child_runs = "tags.mlflow.parentRunId = '{}'".format(run_id)
    runs = mlflow.search_runs(filter_string=filter_child_runs)
    param_cols = ["params.{}".format(p) for p in parameters.keys()]
    metric_cols = ["metrics.mean_test_score"]

    print("\n========== child runs ==========\n")
    pd.set_option("display.max_columns", None)  # prevent truncating columns
    print(runs[["run_id", *param_cols, *metric_cols]])

In [26]:
do_grid_search()

2023/01/18 21:15:48 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b18782a3e95c4d64affd2d47be0950ef', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/01/18 21:15:55 INFO mlflow.sklearn.utils: Logging the 5 best runs, no runs will be omitted.


========== parent run ==========

---------- logged params ----------
{'best_C': '1',
 'best_kernel': 'linear',
 'cv': 'None',
 'error_score': 'nan',
 'estimator': 'SVC()',
 'n_jobs': 'None',
 'param_grid': "{'kernel': ('linear', 'rbf'), 'C': [1, 10]}",
 'pre_dispatch': '2*n_jobs',
 'refit': 'True',
 'return_train_score': 'False',
 'scoring': 'None',
 'verbose': '0'}

---------- logged metrics ----------
{'best_cv_score': 0.9800000000000001,
 'training_accuracy_score': 0.9933333333333333,
 'training_f1_score': 0.9933326665999933,
 'training_precision_score': 0.9934640522875816,
 'training_recall_score': 0.9933333333333333,
 'training_score': 0.9933333333333333}

---------- logged tags ----------
{'estimator_class': 'sklearn.model_selection._search.GridSearchCV',
 'estimator_name': 'GridSearchCV'}

---------- logged artifacts ----------
['cv_results.csv',
 'estimator.html',
 'training_confusion_matrix.png',
 'best_estimator/MLmodel',
 'best_estimator/conda.yaml',
 'best_estimator/model.